In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 绘图全局设置
sns.set_theme(style="whitegrid", font='Times New Roman', font_scale=3)

In [ ]:
# 输出文件路径
PATH, DPI = './export/', 300
# 获取26个英文字母，用于给子图编号
CHAR = [chr(i) for i in range(97, 123)]

In [ ]:
# 读取数据
# size_per_0 = pd.read_excel('./data.xlsx', sheet_name=0)
size_per_0 = pd.read_csv('./data/data.csv')
size_per = size_per_0.copy(deep=True)
size_per

In [ ]:
# 预处理
size_per.set_index('Size (µm)', inplace=True)  # 设置行索引
size_per.dropna(how='all', inplace=True)  # 删除缺失值（行全为空与列全为空）
size_per.dropna(how='all', axis=1, inplace=True)
size_per

In [ ]:
size_per.T

In [ ]:
# 转置
size_per_T = size_per.T
size_per_T

In [ ]:
# 去除掉大粒径颗粒
size_per_T = size_per_T[[i for i in size_per_T.columns if i < 400]]

In [ ]:
# 根据每一行的行索引来为每一行增加Period、River列，例如：行索引为D-N6，那么其Period为Dry Season、River为NanFei River
# size_per_T.drop('Period', axis=1, inplace=True)

size_per_T = size_per_T.copy()  # 当向列表中增加一列时，需要先将变量复制一份，再添加才可以。我看官方文档里都是这么做的，好像是为了防止数据丢失？因为Pandas操作每一步都是不可逆的

# 时期转化参照
# season_paras = {
#     'D': 'Dry Season',
#     'L': 'Level Season',
#     'W': 'Wet Season'
# }
#
# # 地点转化参照
# site_paras = {
#     'N': 'NanFei River',
#     'P': 'Pai River',
#     'H': 'Hangbu River'
# }
#
# for item in season_paras.items():
#     size_per_T.loc[[i for i in size_per_T.index if item[0] in i], 'Period'] = item[1]
#     # 不能使用此种方式修改，语法上没毛病，但是就是不能修改！！！
#     # size_per_T.loc[[i for i in size_per_T.index if item[0] in i],]['Period'] = item[1]
#
# for item in site_paras.items():
#     size_per_T.loc[[i for i in size_per_T.index if item[0] in i], 'River'] = item[1]


size_per_T['Period'] = size_per_T.apply(lambda x: 'Dry' if 'D' in x.name else ('Wet' if 'W' in x.name else 'Level'),
                                        axis=1) + ' Season'
size_per_T['River'] = size_per_T.apply(lambda x: 'Nanfei' if 'N' in x.name else ('Pai' if 'P' in x.name else 'Hangbu'),
                                       axis=1) + ' River'

size_per_T.index.name = 'ID'

size_per_T

In [ ]:
# 按时期分开绘制不同时期的图形
plt.figure(figsize=(60, 20))

# 枯水期底泥粒径百分比分布图
# sns.violinplot()
sns.boxplot(
    data=size_per_T[size_per_T.Period == 'Dry Season'],
    # hue=size_per_T.Period,
    notch=True
)

# 箱型图上加上散点图
# sns.stripplot(data=size_per_T[size_per_T.Period == 'Dry Season'],
#               size=4, color=".3", linewidth=0)

plt.xticks(rotation=90)  # x轴坐标文字旋转
plt.ylabel('Probability of occurrence(%)')  # y轴轴名

# plt.savefig(PATH + '枯水期-底泥粒径百分比分布图', dpi=DPI, bbox_inches='tight')

In [ ]:
# 平水期底泥粒径百分比分布
plt.figure(figsize=(60, 20))

sns.boxplot(
    data=size_per_T[size_per_T.Period == 'Level Season'],
    notch=True)

plt.xticks(rotation=90)  # x轴坐标文字旋转
plt.ylabel('Probability of occurrence(%)')  # y轴轴名

plt.savefig(PATH + '平水期-底泥粒径百分比分布图', dpi=DPI, bbox_inches='tight')

In [ ]:
# 三个时期整体底泥粒径百分比分布
plt.figure(figsize=(60, 20))

sns.boxplot(
    data=size_per_T,
    notch=True)

plt.xticks(rotation=90)  # x轴坐标文字旋转
plt.ylabel('Probability of occurrence(%)')  # y轴轴名

plt.savefig(PATH + '三个时期整体-底泥粒径百分比分布图', dpi=DPI, bbox_inches='tight')

In [ ]:
# 将不同时期的数据绘制到一张表上，使用melt函数将宽数据转化为长数据
size_per_T_long = size_per_T
# size_per_T_long = size_per_T_long.copy()
size_per_T_long['ID'] = size_per_T.index
size_per_T_long = size_per_T_long.melt(id_vars=['ID', 'Period', 'River'])
size_per_T_long

In [ ]:
plt.figure(figsize=(60, 20))

ax = sns.boxplot(
    x="Size (µm)",
    y="value",
    hue="Period",
    data=size_per_T_long,
    notch=True  # 加入参数
)

plt.xticks(rotation=90)  # x轴坐标文字旋转
plt.ylabel('Probability of occurrence(%)')  # y轴轴名
plt.legend(loc='upper left', ncol=2, title='Period')  # 添加图例

plt.savefig(PATH + '三个时期对比-底泥粒径百分比分布图', dpi=DPI, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(60, 20))

sns.boxplot(
    x="Size (µm)",
    y="value",
    hue="River",
    data=size_per_T_long,
    notch=True  # 加入缺口
)

plt.xticks(rotation=90)  # x轴坐标文字旋转
plt.ylabel('Probability of occurrence(%)')  # y轴轴名
plt.legend(loc='upper left', ncol=3, title='River')  # 添加图例

plt.savefig(PATH + '三条河流对比-底泥粒径百分比分布图', dpi=DPI, bbox_inches='tight')

In [ ]:
fig, ax_arr = plt.subplots(2, 1, figsize=(60, 25), sharex='col')

fig.subplots_adjust(hspace=0.05)  # 设置子图的横纵间距

sns.boxplot(
    x="Size (µm)",
    y="value",
    hue="Period",
    data=size_per_T_long,
    notch=True,  # 加入参数
    ax=ax_arr[0]
)

ax_arr[0].set_xlabel(None)  # 注意：子图的属性设置方式与单一图不同，使用ax_arr[0].xticks()则报错
ax_arr[0].legend(loc='upper left', ncol=2, title='Period')  # 添加图例
ax_arr[0].set_ylabel('Probability of occurrence(%)')  # y轴轴名

# sns.violinplot()
sns.boxplot(
    x="Size (µm)",
    y="value",
    hue="River",
    data=size_per_T_long,
    notch=True,  # 加入缺口
    ax=ax_arr[1]
)

plt.xticks(rotation=90)  # x轴坐标文字旋转（这个x轴是整个主图的x轴，因此使用plt.xticks()来设置属性，使用ax_arr[1].set_xlabel()会报错）
ax_arr[1].set_ylabel('Probability of occurrence(%)')  # y轴轴名
ax_arr[1].legend(loc='upper left', ncol=3, title='River')  # 添加图例

plt.savefig(PATH + '季节&位点对比-底泥粒径百分比分布图', dpi=DPI, bbox_inches='tight')

In [ ]:
size_per_T_long

In [ ]:
season_list = size_per_T_long.Period.unique().tolist()
season_list

In [ ]:
fig, ax_arr = plt.subplots(len(season_list), 1, figsize=(60, 40), sharex='col')

fig.subplots_adjust(hspace=0.05)  # 设置子图的横纵间距

for i in range(len(ax_arr)):
    item = sns.boxplot(
        x="Size (µm)",
        y="value",
        hue="River",
        data=size_per_T_long[size_per_T_long['Period'] == season_list[i]],
        notch=True,  # 加入缺口
        ax=ax_arr[i]
    )

    # 删除子图的图例
    item.get_legend().remove()
    # 子图添加编号
    item.text(
        0.02,
        0.9,
        f"({CHAR[i]}) - {season_list[i]}",
        transform=item.transAxes,  # 默认的话前两个参数是(x, y)坐标，设置该属性后是相较于x、y轴的比例
        # fontdict={'size': '16', 'color': 'b'}
    )
    # 设置子图x轴名称，只保留最后一个子图的轴名称，其余删除
    if i != len(ax_arr)-1:
        item.set_xlabel(None)
    else:
        item.set_xlabel('Particle size(μm)')
    # 设置子图y轴名称
    item.set_ylabel('Percentage(%)')

plt.xticks(rotation=90)  # x轴坐标文字旋转（这个x轴是整个主图的x轴，因此使用plt.xticks()来设置属性，使用ax_arr[1].set_xlabel()会报错）
# 添加图例
lines, labels = fig.axes[-1].get_legend_handles_labels()  # 三个子图的图例相同，获取最后一个子图的图例
fig.legend(lines, labels, ncol=3, loc='upper center', bbox_to_anchor=(0.5, 0.91))
# 保存
plt.savefig(PATH + '三个时期每条河流粒径分布百分比图', dpi=DPI, bbox_inches='tight')
# 打开保存文件夹
os.startfile(os.getcwd() + PATH)

In [ ]:
# 打开保存文件夹
os.startfile(os.getcwd() + PATH)